In [1]:
%config Completer.use_jedi = False
import configparser
config = configparser.ConfigParser()
config.read('XXX-config.txt')

source_folder = config.get('Configuration', 'source_folder')
time_step = int(config.get('Configuration', 'time_step'))
window_size = int(config.get('Configuration', 'window_size'))
data_folder = config.get('Configuration', 'data_folder')
ini_date = config.get('Configuration', 'ini_date')
end_date = config.get('Configuration', 'end_date')

users=config.get('Configuration', 'users')
users = [(item.strip()) for item in users.split(',')]

w = int(config.get('Configuration', 'w'))
h = int(config.get('Configuration', 'h'))

anchors=config.get('Configuration', 'anchors')
anchors = [(item.strip()) for item in anchors.split(',')]

rooms=config.get('Configuration', 'rooms')
rooms = [(item.strip()) for item in rooms.split(',')]

In [2]:
import pandas as pd
import collections
import math
import os

import cv2
import matplotlib.pyplot as plt
import cv2
import sys
import numpy as np
import math
import time
from PIL import Image 
import shutil

from pandas import read_csv  



In [3]:
import numpy as np
from pandas import read_csv    
import pandas as pd
import time

from datetime import datetime

off_zone=60*60*2

def day_time(ti):
    return int((int)((ti+off_zone)/(60*60*24)))


def time2str(tt):
    return datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S")


t0 = (int)(datetime.strptime(ini_date, '%Y-%m-%d %H:%M:%S').timestamp())
print (ini_date,"Init date is", t0, "day:",day_time(t0), time2str(t0))

tN = (int)(datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S').timestamp())
print (end_date,"End date is", tN, "day:",day_time(tN), time2str(tN))


def day_time0(ti):
    return day_time(ti)-day_time(t0)+1

def relT(ti):
    return (int)((ti-t0)/time_step)

#day=list(range(day_time(t0),day_time(tN)+1))[0]
day=19798
print(day)

1980-01-01 13:00:02 Init date is 315576002 day: 3652 1980-01-01 13:00:02
1980-01-01 14:51:20 End date is 315582680 day: 3652 1980-01-01 14:51:20
19798


In [4]:
def getLoc(room,s):
    print("S:",s)
    xy=config.get(room, s)
    xy = [(item.strip()) for item in xy.split(',')]
    xy = [list(map(int, item.split('x'))) for item in xy]
    return xy

ix = {}
iy = {}

offW= {}
offH={}

for room in rooms:
    print(room)
    ix[room] = int(config.get(room, 'ix'))
    iy[room] = int(config.get(room, 'iy'))

    offW[room] = int(config.get(room, 'offW'))
    offH[room] = int(config.get(room, 'offH'))    

livingroom
kitchen
bathroom


In [5]:
df_total_map = {}

for room in rooms:
    df_total_map[room]={}
    print(os.listdir(os.path.join(source_folder,room)))
    for user in users:
        df_total_map[room][user]=None
        
    for person in os.listdir(os.path.join(source_folder,room)):
        for scene in os.listdir(os.path.join(source_folder,room,person)):
            print("scene",scene)
            for user in users:   
                print(os.path.join(source_folder,room,person,scene)+"/location_gt_"+user+"/"+str(day)+".location.tsv")
                try:
                    series=read_csv(os.path.join(source_folder,room,person,scene)+"/location_gt_"+user+"/"+str(day)+".location.tsv", sep="\t",parse_dates=True, header=None, names=["time","date","time2","x","y"])
                    print(series.size,(series["time"].iloc[0],series["time"].iloc[-1]))
                except Exception as e:
                    print("Not data",e)
                    continue
                if(df_total_map[room][user] is None):
                    df_total_map[room][user]=series
                else:
                    df_total_map[room][user] = df_total_map[room][user].append(series, ignore_index=True)


['1-person-labelled', '2-person-labelled', '3-person-labelled']
scene Scene1
./data/-aligned/livingroom\1-person-labelled\Scene1/location_gt_f02e/19798.location.tsv
Not data [Errno 2] No such file or directory: './data/-aligned/livingroom\\1-person-labelled\\Scene1/location_gt_f02e/19798.location.tsv'
./data/-aligned/livingroom\1-person-labelled\Scene1/location_gt_867f/19798.location.tsv
Not data [Errno 2] No such file or directory: './data/-aligned/livingroom\\1-person-labelled\\Scene1/location_gt_867f/19798.location.tsv'
./data/-aligned/livingroom\1-person-labelled\Scene1/location_gt_ed9c/19798.location.tsv
1020 (315576000, 315576294)
scene Scene2
./data/-aligned/livingroom\1-person-labelled\Scene2/location_gt_f02e/19798.location.tsv
Not data [Errno 2] No such file or directory: './data/-aligned/livingroom\\1-person-labelled\\Scene2/location_gt_f02e/19798.location.tsv'
./data/-aligned/livingroom\1-person-labelled\Scene2/location_gt_867f/19798.location.tsv
Not data [Errno 2] No such f

C:\Users\javie\AppData\Local\Temp\ipykernel_34508\434904795.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total_map[room][user] = df_total_map[room][user].append(series, ignore_index=True)
C:\Users\javie\AppData\Local\Temp\ipykernel_34508\434904795.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total_map[room][user] = df_total_map[room][user].append(series, ignore_index=True)
C:\Users\javie\AppData\Local\Temp\ipykernel_34508\434904795.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total_map[room][user] = df_total_map[room][user].append(series, ignore_index=True)
C:\Users\javie\AppData\Local\Temp\ipykernel_34508\434904795.py:23: FutureWarning: The frame.append method is deprecated and will be remo

In [6]:
def getXY(x,y,ix,iy,offx,offy):
    #print("x,y",x,y)
    x0=int(x/ix)
    y0=int(y/iy)
    #print("x0,y0",x0,y0)
    x0=x0+offx    
    y0=y0+offy
    #print(x0,y0)
    if(x0>=h):
        x0=h-1
    if(y0>=w):
        y0=w-1
    return x0,y0

In [7]:
for user in users:
    last_df_intervalo={}
    for room in rooms:
        last_df_intervalo[room]=None
    for tt in range(t0,tN,time_step):
        
        #if(day_time(tt) != day):
        #    continue

        img0=np.zeros((h, w))
        for room in rooms:
            print(user, room, tt,time2str(tt))
            df_total=df_total_map[room][user]
            if(df_total is None):
                continue
            df_intervalo = df_total[(df_total['time'] >= (tt-window_size/2)) & (df_total['time'] <= (tt))]
            #print("DF",df_intervalo)     
            if df_intervalo.empty:
                #print("Zero GT complete with last interval",df_intervalo)   
                df_intervalo=last_df_intervalo[room]
            else:
                last_df_intervalo[room]=df_intervalo
            if not(df_intervalo is None or df_intervalo.empty):
                for index, row in df_intervalo.iterrows():
                    dif=abs(tt-row['time'])+1
                    #print("@TIME",tt,row, row['time'], "dif:", dif, "xy:", row['x'],row['y'])
                    #print("row['x'],row['y']",row['x'],row['y'])
                    xy=getXY(row['y'],row['x'],ix[room],iy[room],offH[room],offW[room])

                    #print(room,"xy",xy)
                    #print("#xy",offx[room],offy[room])
                    img0[xy[0]][xy[1]]=img0[xy[0]][xy[1]]+int(255/dif)
        img0[img0>255]=255
        cv2.imwrite(data_folder+"/DAY_"+str(day)+"/GT."+user+"."+str(tt)+".png", img0) 

f02e livingroom 315576002 1980-01-01 13:00:02
f02e kitchen 315576002 1980-01-01 13:00:02
f02e bathroom 315576002 1980-01-01 13:00:02
f02e livingroom 315576003 1980-01-01 13:00:03
f02e kitchen 315576003 1980-01-01 13:00:03
f02e bathroom 315576003 1980-01-01 13:00:03
f02e livingroom 315576004 1980-01-01 13:00:04
f02e kitchen 315576004 1980-01-01 13:00:04
f02e bathroom 315576004 1980-01-01 13:00:04
f02e livingroom 315576005 1980-01-01 13:00:05
f02e kitchen 315576005 1980-01-01 13:00:05
f02e bathroom 315576005 1980-01-01 13:00:05
f02e livingroom 315576006 1980-01-01 13:00:06
f02e kitchen 315576006 1980-01-01 13:00:06
f02e bathroom 315576006 1980-01-01 13:00:06
f02e livingroom 315576007 1980-01-01 13:00:07
f02e kitchen 315576007 1980-01-01 13:00:07
f02e bathroom 315576007 1980-01-01 13:00:07
f02e livingroom 315576008 1980-01-01 13:00:08
f02e kitchen 315576008 1980-01-01 13:00:08
f02e bathroom 315576008 1980-01-01 13:00:08
f02e livingroom 315576009 1980-01-01 13:00:09
f02e kitchen 315576009 

f02e livingroom 315576066 1980-01-01 13:01:06
f02e kitchen 315576066 1980-01-01 13:01:06
f02e bathroom 315576066 1980-01-01 13:01:06
f02e livingroom 315576067 1980-01-01 13:01:07
f02e kitchen 315576067 1980-01-01 13:01:07
f02e bathroom 315576067 1980-01-01 13:01:07
f02e livingroom 315576068 1980-01-01 13:01:08
f02e kitchen 315576068 1980-01-01 13:01:08
f02e bathroom 315576068 1980-01-01 13:01:08
f02e livingroom 315576069 1980-01-01 13:01:09
f02e kitchen 315576069 1980-01-01 13:01:09
f02e bathroom 315576069 1980-01-01 13:01:09
f02e livingroom 315576070 1980-01-01 13:01:10
f02e kitchen 315576070 1980-01-01 13:01:10
f02e bathroom 315576070 1980-01-01 13:01:10
f02e livingroom 315576071 1980-01-01 13:01:11
f02e kitchen 315576071 1980-01-01 13:01:11
f02e bathroom 315576071 1980-01-01 13:01:11
f02e livingroom 315576072 1980-01-01 13:01:12
f02e kitchen 315576072 1980-01-01 13:01:12
f02e bathroom 315576072 1980-01-01 13:01:12
f02e livingroom 315576073 1980-01-01 13:01:13
f02e kitchen 315576073 

f02e livingroom 315576159 1980-01-01 13:02:39
f02e kitchen 315576159 1980-01-01 13:02:39
f02e bathroom 315576159 1980-01-01 13:02:39
f02e livingroom 315576160 1980-01-01 13:02:40
f02e kitchen 315576160 1980-01-01 13:02:40
f02e bathroom 315576160 1980-01-01 13:02:40
f02e livingroom 315576161 1980-01-01 13:02:41
f02e kitchen 315576161 1980-01-01 13:02:41
f02e bathroom 315576161 1980-01-01 13:02:41
f02e livingroom 315576162 1980-01-01 13:02:42
f02e kitchen 315576162 1980-01-01 13:02:42
f02e bathroom 315576162 1980-01-01 13:02:42
f02e livingroom 315576163 1980-01-01 13:02:43
f02e kitchen 315576163 1980-01-01 13:02:43
f02e bathroom 315576163 1980-01-01 13:02:43
f02e livingroom 315576164 1980-01-01 13:02:44
f02e kitchen 315576164 1980-01-01 13:02:44
f02e bathroom 315576164 1980-01-01 13:02:44
f02e livingroom 315576165 1980-01-01 13:02:45
f02e kitchen 315576165 1980-01-01 13:02:45
f02e bathroom 315576165 1980-01-01 13:02:45
f02e livingroom 315576166 1980-01-01 13:02:46
f02e kitchen 315576166 

f02e livingroom 315576233 1980-01-01 13:03:53
f02e kitchen 315576233 1980-01-01 13:03:53
f02e bathroom 315576233 1980-01-01 13:03:53
f02e livingroom 315576234 1980-01-01 13:03:54
f02e kitchen 315576234 1980-01-01 13:03:54
f02e bathroom 315576234 1980-01-01 13:03:54
f02e livingroom 315576235 1980-01-01 13:03:55
f02e kitchen 315576235 1980-01-01 13:03:55
f02e bathroom 315576235 1980-01-01 13:03:55
f02e livingroom 315576236 1980-01-01 13:03:56
f02e kitchen 315576236 1980-01-01 13:03:56
f02e bathroom 315576236 1980-01-01 13:03:56
f02e livingroom 315576237 1980-01-01 13:03:57
f02e kitchen 315576237 1980-01-01 13:03:57
f02e bathroom 315576237 1980-01-01 13:03:57
f02e livingroom 315576238 1980-01-01 13:03:58
f02e kitchen 315576238 1980-01-01 13:03:58
f02e bathroom 315576238 1980-01-01 13:03:58
f02e livingroom 315576239 1980-01-01 13:03:59
f02e kitchen 315576239 1980-01-01 13:03:59
f02e bathroom 315576239 1980-01-01 13:03:59
f02e livingroom 315576240 1980-01-01 13:04:00
f02e kitchen 315576240 

f02e livingroom 315576335 1980-01-01 13:05:35
f02e kitchen 315576335 1980-01-01 13:05:35
f02e bathroom 315576335 1980-01-01 13:05:35
f02e livingroom 315576336 1980-01-01 13:05:36
f02e kitchen 315576336 1980-01-01 13:05:36
f02e bathroom 315576336 1980-01-01 13:05:36
f02e livingroom 315576337 1980-01-01 13:05:37
f02e kitchen 315576337 1980-01-01 13:05:37
f02e bathroom 315576337 1980-01-01 13:05:37
f02e livingroom 315576338 1980-01-01 13:05:38
f02e kitchen 315576338 1980-01-01 13:05:38
f02e bathroom 315576338 1980-01-01 13:05:38
f02e livingroom 315576339 1980-01-01 13:05:39
f02e kitchen 315576339 1980-01-01 13:05:39
f02e bathroom 315576339 1980-01-01 13:05:39
f02e livingroom 315576340 1980-01-01 13:05:40
f02e kitchen 315576340 1980-01-01 13:05:40
f02e bathroom 315576340 1980-01-01 13:05:40
f02e livingroom 315576341 1980-01-01 13:05:41
f02e kitchen 315576341 1980-01-01 13:05:41
f02e bathroom 315576341 1980-01-01 13:05:41
f02e livingroom 315576342 1980-01-01 13:05:42
f02e kitchen 315576342 

...

ed9c bathroom 315582163 1980-01-01 14:42:43
ed9c livingroom 315582164 1980-01-01 14:42:44
ed9c kitchen 315582164 1980-01-01 14:42:44
ed9c bathroom 315582164 1980-01-01 14:42:44
ed9c livingroom 315582165 1980-01-01 14:42:45
ed9c kitchen 315582165 1980-01-01 14:42:45
ed9c bathroom 315582165 1980-01-01 14:42:45
ed9c livingroom 315582166 1980-01-01 14:42:46
ed9c kitchen 315582166 1980-01-01 14:42:46
ed9c bathroom 315582166 1980-01-01 14:42:46
ed9c livingroom 315582167 1980-01-01 14:42:47
ed9c kitchen 315582167 1980-01-01 14:42:47
ed9c bathroom 315582167 1980-01-01 14:42:47
ed9c livingroom 315582168 1980-01-01 14:42:48
ed9c kitchen 315582168 1980-01-01 14:42:48
ed9c bathroom 315582168 1980-01-01 14:42:48
ed9c livingroom 315582169 1980-01-01 14:42:49
ed9c kitchen 315582169 1980-01-01 14:42:49
ed9c bathroom 315582169 1980-01-01 14:42:49
ed9c livingroom 315582170 1980-01-01 14:42:50
ed9c kitchen 315582170 1980-01-01 14:42:50
ed9c bathroom 315582170 1980-01-01 14:42:50
ed9c livingroom 315582171

ed9c livingroom 315582267 1980-01-01 14:44:27
ed9c kitchen 315582267 1980-01-01 14:44:27
ed9c bathroom 315582267 1980-01-01 14:44:27
ed9c livingroom 315582268 1980-01-01 14:44:28
ed9c kitchen 315582268 1980-01-01 14:44:28
ed9c bathroom 315582268 1980-01-01 14:44:28
ed9c livingroom 315582269 1980-01-01 14:44:29
ed9c kitchen 315582269 1980-01-01 14:44:29
ed9c bathroom 315582269 1980-01-01 14:44:29
ed9c livingroom 315582270 1980-01-01 14:44:30
ed9c kitchen 315582270 1980-01-01 14:44:30
ed9c bathroom 315582270 1980-01-01 14:44:30
ed9c livingroom 315582271 1980-01-01 14:44:31
ed9c kitchen 315582271 1980-01-01 14:44:31
ed9c bathroom 315582271 1980-01-01 14:44:31
ed9c livingroom 315582272 1980-01-01 14:44:32
ed9c kitchen 315582272 1980-01-01 14:44:32
ed9c bathroom 315582272 1980-01-01 14:44:32
ed9c livingroom 315582273 1980-01-01 14:44:33
ed9c kitchen 315582273 1980-01-01 14:44:33
ed9c bathroom 315582273 1980-01-01 14:44:33
ed9c livingroom 315582274 1980-01-01 14:44:34
ed9c kitchen 315582274 

ed9c kitchen 315582381 1980-01-01 14:46:21
ed9c bathroom 315582381 1980-01-01 14:46:21
ed9c livingroom 315582382 1980-01-01 14:46:22
ed9c kitchen 315582382 1980-01-01 14:46:22
ed9c bathroom 315582382 1980-01-01 14:46:22
ed9c livingroom 315582383 1980-01-01 14:46:23
ed9c kitchen 315582383 1980-01-01 14:46:23
ed9c bathroom 315582383 1980-01-01 14:46:23
ed9c livingroom 315582384 1980-01-01 14:46:24
ed9c kitchen 315582384 1980-01-01 14:46:24
ed9c bathroom 315582384 1980-01-01 14:46:24
ed9c livingroom 315582385 1980-01-01 14:46:25
ed9c kitchen 315582385 1980-01-01 14:46:25
ed9c bathroom 315582385 1980-01-01 14:46:25
ed9c livingroom 315582386 1980-01-01 14:46:26
ed9c kitchen 315582386 1980-01-01 14:46:26
ed9c bathroom 315582386 1980-01-01 14:46:26
ed9c livingroom 315582387 1980-01-01 14:46:27
ed9c kitchen 315582387 1980-01-01 14:46:27
ed9c bathroom 315582387 1980-01-01 14:46:27
ed9c livingroom 315582388 1980-01-01 14:46:28
ed9c kitchen 315582388 1980-01-01 14:46:28
ed9c bathroom 315582388 19

ed9c bathroom 315582493 1980-01-01 14:48:13
ed9c livingroom 315582494 1980-01-01 14:48:14
ed9c kitchen 315582494 1980-01-01 14:48:14
ed9c bathroom 315582494 1980-01-01 14:48:14
ed9c livingroom 315582495 1980-01-01 14:48:15
ed9c kitchen 315582495 1980-01-01 14:48:15
ed9c bathroom 315582495 1980-01-01 14:48:15
ed9c livingroom 315582496 1980-01-01 14:48:16
ed9c kitchen 315582496 1980-01-01 14:48:16
ed9c bathroom 315582496 1980-01-01 14:48:16
ed9c livingroom 315582497 1980-01-01 14:48:17
ed9c kitchen 315582497 1980-01-01 14:48:17
ed9c bathroom 315582497 1980-01-01 14:48:17
ed9c livingroom 315582498 1980-01-01 14:48:18
ed9c kitchen 315582498 1980-01-01 14:48:18
ed9c bathroom 315582498 1980-01-01 14:48:18
ed9c livingroom 315582499 1980-01-01 14:48:19
ed9c kitchen 315582499 1980-01-01 14:48:19
ed9c bathroom 315582499 1980-01-01 14:48:19
ed9c livingroom 315582500 1980-01-01 14:48:20
ed9c kitchen 315582500 1980-01-01 14:48:20
ed9c bathroom 315582500 1980-01-01 14:48:20
ed9c livingroom 315582501

ed9c kitchen 315582597 1980-01-01 14:49:57
ed9c bathroom 315582597 1980-01-01 14:49:57
ed9c livingroom 315582598 1980-01-01 14:49:58
ed9c kitchen 315582598 1980-01-01 14:49:58
ed9c bathroom 315582598 1980-01-01 14:49:58
ed9c livingroom 315582599 1980-01-01 14:49:59
ed9c kitchen 315582599 1980-01-01 14:49:59
ed9c bathroom 315582599 1980-01-01 14:49:59
ed9c livingroom 315582600 1980-01-01 14:50:00
ed9c kitchen 315582600 1980-01-01 14:50:00
ed9c bathroom 315582600 1980-01-01 14:50:00
ed9c livingroom 315582601 1980-01-01 14:50:01
ed9c kitchen 315582601 1980-01-01 14:50:01
ed9c bathroom 315582601 1980-01-01 14:50:01
ed9c livingroom 315582602 1980-01-01 14:50:02
ed9c kitchen 315582602 1980-01-01 14:50:02
ed9c bathroom 315582602 1980-01-01 14:50:02
ed9c livingroom 315582603 1980-01-01 14:50:03
ed9c kitchen 315582603 1980-01-01 14:50:03
ed9c bathroom 315582603 1980-01-01 14:50:03
ed9c livingroom 315582604 1980-01-01 14:50:04
ed9c kitchen 315582604 1980-01-01 14:50:04
ed9c bathroom 315582604 19

In [8]:
x0,y0=100,532
room="bathroom"
xy=getXY(x0,y0,ix[room],iy[room],offH[room],offW[room])
print(xy)

(9, 11)
